# The RGB slope - metallicity relation (Valenti et al. 2004)
In this notebook we will reproduce some of the results from [Valenti et al. (2004)](https://doi.org/10.1111/j.1365-2966.2004.07861.x),
a paper that presents near-infrared photometry (in the J, H, and K band) of 10 Galactic globular clusters.
The paper was published in the Monthly Notices of the Royal Astronomical Society (MNRAS).

We will download the data used to make the analysis and reproduce some of the plots and the results in the paper, in particular the correlation
between the slope of the Red Giant Branch (RGB) and the metallicity of globular clusters.
** the RGB slope is defined in the K, (J–K) plane**, not in the color-magnitude diagram (J-K), K [read the paper if you want details]


To this end, the steps to be carrried out are:
* download the complete photometric catalogue for the 10 cluster.
* use one of the clusters as a test case to develop a procedure to calculate the slope of the RGB
* convert this procedure into a function to be applied to all clusters
* check the slope/metallicity relation

This notebook will show how to:
* Download data from the Virtual Observatory, using [Vizier](http://vizier.u-strasbg.fr/viz-bin/VizieR)
* Get data from a tabular ascii file
* select data relevant for the analysis
* make a linear fit to the data
* define a function to carry out all calculations and apply it to a number of different cases [each one of the clusters]



In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

The metallicity [Fe/H] of the 10 clusters are listed in the file ascii `cluster.dat`. The data were taken from Table 1 in the paper.

Load the data into an `astropy` `Table`

In [2]:
from astropy.io import ascii

clusters=ascii.read("clusters.dat")
clusters

cluster,FeH
str5,float64
M15,-2.12
M30,-1.91
N288,-1.07
N362,-1.15
N6342,-0.71
N6380,-0.87
N6440,-0.49
N6441,-0.68
N6624,-0.63


# Download data from vizier
The photometric catalogue used in the paper is available here:
http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/MNRAS/351/1204/phot

The `astroquery` package provides a python interface for querying the [VizieR](http://vizier.u-strasbg.fr/) web service, the most complete library of published astronomical catalogues and data tables available on line.

`Astroquery` is a package affiliated to astropy [more details on affiliated packages here: http://www.astropy.org/affiliated/index.html]

In [3]:
from astroquery.vizier import Vizier

# create the Vizier object:
v=Vizier()

# and remove the limit to number of rows to be retrieved
# by default vizier return only the first 50 row of a catalogue!
v.ROW_LIMIT=-1

On the Vizier page where you have the [catalog from the paper](http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/MNRAS/351/1204/phot) you can find 
that
the code of this catalog is `J/MNRAS/351/1204/phot`.

To download the data we will use the `get_catalogs()` function.
Note that this function accept in input a list of catalogs, if you give only one catalog name in input, you will get a list with only one catalog,
to get the catalog itself, you need to select the first (and only) element of the output list.


In [4]:
# get the catalog
catalog0= v.get_catalogs("J/MNRAS/351/1204/phot")[0]

catalog0.info()

<Table masked=True length=22248>
  name   dtype  unit  format                   description                  n_bad
------- ------- ---- -------- --------------------------------------------- -----
   GlCl  bytes5                                  Cluster name ('N' for NGC)     0
    Seq   int16                            Sequential number in the cluster     0
   Jmag float32  mag  {:6.3f}                           ? IRAC2/J magnitude  1690
   Hmag float32  mag  {:6.3f}                           ? IRAC2/H magnitude  2169
   Kmag float32  mag  {:6.3f}                           ? IRAC2/K magnitude  3438
RAJ2000 float64  deg {:10.6f} Right Ascension J2000, Epoch 1989.0 (degrees)     0
DEJ2000 float64  deg {:10.6f}     Declination J2000, Epoch 1989.0 (degrees)     0


The output is an astropy `Table`.

In [5]:
catalog0

GlCl,Seq,Jmag,Hmag,Kmag,RAJ2000,DEJ2000
,,mag,mag,mag,deg,deg
bytes5,int16,float32,float32,float32,float64,float64
M15,1,10.196,9.548,9.419,322.493072,12.162783
M15,2,10.217,9.564,9.437,322.509369,12.189307
M15,3,10.265,9.671,9.545,322.486572,12.174149
M15,4,10.268,9.910,9.661,322.494019,12.155949
M15,5,10.285,10.015,9.918,322.489837,12.162182
M15,6,10.294,--,9.511,322.494019,12.155974
M15,7,10.327,9.718,9.612,322.484070,12.171647
M15,8,10.351,9.708,9.581,322.497131,12.153227


Some of the values in the original catalog are `nan` (because some of the magnitudes of some stars could not be measured). 
Each column in an `astropy` `Table` has a `mask`. This is a boolean numpy array that is True when the original data were `nan`. 

In [6]:
Jmask=catalog0["Jmag"].mask
Kmask=catalog0["Kmag"].mask
print (Jmask)
print("stars with masked Jmag",Jmask.sum())
print("stars with masked Kmag",Kmask.sum())


[False False False ..., False False False]
stars with masked Jmag 1690
stars with masked Kmag 3438


In [7]:
# select only rows with valid J and K mags
# i.e. both Jmag and Kmag mask are False

good=(Jmask==False)&(Kmask==False)
catalog=catalog0[good]

len(catalog)


17120

# Use one cluster as a test case

* choose a cluster, eg, M15
* select the J and K magnitude for the stars of your cluster
* plot the (J-K) vs K color-magnitude diagram to have an idea of what the data look like
* select stars in a magnitude range between 0.5 and 5 mag fainter than the brightest star [to follow the selection done in the paper]
* overplot these bright stars on the CMD using a different color

In [8]:
cname="M15"

# create a condition to select only data for one cluster

# select the columns with J and K-band magnitudes and get only data for your cluster

# J=
# K=
# JK=J-K

# create a condition to select only the brightest stars 

# plot the CMD


In [9]:
# %load solutions/part1.py

## Fit a straight line to the bright-end of the RGB in the K, J-K plane


In [10]:
# make the fit

# create the best fit line

# overplot on the CMD

In [11]:
# %load solutions/part2.py

# put it all togheter in a function 
* the name of the cluster as input
* create a file with the plot of the fit, so you can check that the fit is OK
* return the slope

In [12]:
# create a directory for the plots

# define a function to measure the slope of the RGB and plot the results
def get_slope(cname):
    pass

In [ ]:
# %load solutions/function_slope.py

In [ ]:
# check that everithing works
get_slope("M15")

# iterate on all obects

In [ ]:

# add a column called slope with a non-sense number, then we will fill it with the value we calculate
clusters["slope"]=-999.9 # it is important to have floats!

# iterate
for row in clusters:
    s=get_slope(row["cluster"])
    row["slope"]=s

clusters

In [ ]:
xfit=np.linspace(-.114,-.035,2)
yfit_valenti=-22.21*xfit-2.80 # fit from Valenti et al 2004, Figure 11

p=np.polyfit(clusters["slope"],clusters["FeH"],1)
yfit_me=np.polyval(p,xfit)

fig,ax=plt.subplots()
ax.plot(clusters["slope"],clusters["FeH"].data,'ok')
ax.plot(xfit,yfit_valenti,ls=":")
ax.plot(xfit,yfit_me)

ax.set_xlim(-0.03,-0.119)
ax.set_ylim(-2.5,.4)
